In [1]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# agent使用入门

In [2]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import load_tools
from langchain.agents import create_openai_functions_agent  #不同的agent有不同的创建方式
from langchain.agents import AgentExecutor

#创建LLM 
llm = ChatOpenAI(model_name="gpt-4", temperature=0)
#定义agent的prompt:可以自己编写,也可以从hub拉取
#https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")
#定义工具,加载预制的工具,注意有的工具需要提供LLM
tools = load_tools(["llm-math"], llm=llm)
#创建agent
agent = create_openai_functions_agent(llm, tools, prompt)
#定义agent的执行器，这里注意与老版本的不同
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "你好"})



> Entering new AgentExecutor chain...
你好！有什么我可以帮助你的吗？

> Finished chain.


{'input': '你好', 'output': '你好！有什么我可以帮助你的吗？'}

# 自定义tool+ 使用系统tool

## 自定义tool

In [27]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_community.utilities import SerpAPIWrapper
import os

os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"


#使用注解@tool即可将一个方法变成工具
#方法第一行是tools的使用描述
@tool
def search(query: str) -> str:
    """当需要查找实时信息的时候才会使用这个工具."""
    serp = SerpAPIWrapper()
    return serp.run(query)



In [28]:
search("北京今天的天气如何?")

"{'type': 'weather_result', 'temperature': '72', 'unit': 'Fahrenheit', 'precipitation': '20%', 'humidity': '55%', 'wind': '4 mph', 'location': '中国北京市', 'date': 'Saturday', 'weather': 'Cloudy'}"

## 调用系统tool - create_retriever_tool

In [30]:
#RAG增强生成
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 会把该网页内容拿下来存储到向量数据库中
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()
print(f"Loaded {len(docs)} documents")
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()
#搜索匹配文档块
retriever.invoke("如何debug?")[0]

Loaded 1 documents


Document(page_content='Oftentimes, it isn‚Äôt necessary to look at every single trace. However, when things go wrong (an unexpected end result, infinite agent loop, slower than expected execution, higher than expected token usage), it‚Äôs extremely helpful to debug by looking through the application traces. LangSmith gives clear visibility and debugging information at each step of an LLM sequence, making it much easier to identify and root-cause issues.\nWe provide native rendering of chat messages, functions, and retrieve documents.Initial Test Set‚ÄãWhile many developers still ship an initial version of their application based on ‚Äúvibe checks‚Äù, we‚Äôve seen an increasing number of engineering teams start to adopt a more test driven approach. LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSm

In [31]:
#把检索器加入到工具中
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="langsmith_search",
    description="搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！",
)
retriever_tool

Tool(name='langsmith_search', description='搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x117c7c180>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12750b1d0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x117c7c220>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12750b1d0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [32]:
#可用工具集
tools = [search, retriever_tool]
tools

[StructuredTool(name='search', description='search(query: str) -> str - 当需要查找实时信息的时候才会使用这个工具.', args_schema=<class 'pydantic.main.searchSchema'>, func=<function search at 0x127df31a0>),
 Tool(name='langsmith_search', description='搜索有关 LangSmith 的信息。关于LangSmith的任何问题，你一定要使用这个工具！', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x117c7c180>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12750b1d0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x117c7c220>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x12750b1d0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_

## agent带哦用

In [33]:
#从hub中获取模版
from langchain import hub
#创建agent
from langchain.agents import create_openai_functions_agent
#创建agent执行器AgentExecutor
from langchain.agents import AgentExecutor

#配置模型
model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
# 一个最简单的模版,带记忆
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [34]:
#执行
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [35]:
agent_executor.invoke({"input": "langsmith如何帮助做项目测试?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': '如何帮助做项目测试'}`


Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing‚ÄãBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it‚Äôs important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it‚Äôs breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback‚ÄãWhen launching your application to an initial set of users, it‚Äôs important to gather human feedback on the responses it‚Äôs producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic respo

{'input': 'langsmith如何帮助做项目测试?',
 'output': 'LangSmith提供了一系列的工具和功能来帮助进行项目测试：\n\n1. **日志记录**：每次在playground运行的日志都会被系统记录下来，这些日志可以用来创建测试用例或与其他运行进行比较。\n\n2. **Beta测试**：Beta测试允许开发者在真实世界的场景中收集他们的LLM应用程序的性能数据。在这个阶段，理解应用在哪些输入上表现良好或不良，以及在哪些情况下出现问题是非常重要的。反馈收集和运行注释对于这个工作流程至关重要。这将有助于策划测试用例，帮助跟踪回归/改进和开发自动评估。\n\n3. **反馈收集**：在向初始用户群发布应用程序时，收集人类对其产生的反应的反馈是非常重要的。这有助于引起对最有趣运行的关注，并突出导致问题反应的边缘案例。LangSmith允许你将反馈分数附加到记录的追踪中。\n\n4. **比较视图**：在原型化你的应用程序的不同版本并进行更改时，看看你是否在初始测试用例上有所回归是非常重要的。在提示、检索策略或模型选择的变化可能会对应用程序产生的反应产生巨大影响。为了得到哪个变体表现更好的感觉，能够在同一数据点上查看不同配置的结果是非常有用的。我们在测试运行的比较视图上投入了大量的精力，以便跟踪和诊断在应用程序的多个修订版中的测试分数的回归。\n\n5. **Playground**：LangSmith提供了一个playground环境，用于快速迭代和实验。\n\n6. **数据集**：在应用程序进入beta测试阶段时，继续收集数据以精炼和提高其性能是至关重要的。LangSmith允许你将运行作为示例添加到数据集中，扩大你在真实世界场景中的测试覆盖范围。\n\n7. **生产**：在应用程序进入生产阶段时，仔细检查关键数据点，增长基准数据集，注释追踪，深入重要数据的追踪视图是你也想做的工作流程。然而，尤其是在生产阶段，获取应用程序性能的高级概述，包括延迟、成本和反馈分数，是至关重要的。这确保了它在大规模下提供了理想的结果。\n\n8. **初始测试集**：LangSmith允许开发者创建数据集，这些数据集是输入和参考输出的集合，并使用这些来对他们的LLM应用程序进行测试。\n\n总的来说，LangSmith提供了一套完整的工具和功能，帮助

In [36]:
#实时信息提问
agent_executor.invoke({"input": "北京今天的天气如何?"})



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': '北京今天的天气'}`


{'type': 'weather_result', 'temperature': '72', 'unit': 'Fahrenheit', 'precipitation': '20%', 'humidity': '55%', 'wind': '4 mph', 'location': '中国北京市', 'date': 'Saturday', 'weather': 'Cloudy'}北京今天的天气是多云的，温度为72华氏度，降水概率为20%，湿度为55%，风速为4英里/小时。

> Finished chain.


{'input': '北京今天的天气如何?',
 'output': '北京今天的天气是多云的，温度为72华氏度，降水概率为20%，湿度为55%，风速为4英里/小时。'}

In [37]:
#前面的交互都不带记忆
agent_executor.invoke({"input": "截止目前我们都聊了什么?"})



> Entering new AgentExecutor chain...
对不起，我无法回答这个问题，因为我没有能力回顾我们之前的对话。我是一个基于AI的助手，我只能回答你当前的问题或者提供你需要的信息。如果你有任何问题，欢迎随时向我提问。

> Finished chain.


{'input': '截止目前我们都聊了什么?',
 'output': '对不起，我无法回答这个问题，因为我没有能力回顾我们之前的对话。我是一个基于AI的助手，我只能回答你当前的问题或者提供你需要的信息。如果你有任何问题，欢迎随时向我提问。'}

## 添加记忆

In [38]:
#交互时添加记忆
agent_executor.invoke({"input": "hi! 我叫bob", "chat_history": []})



> Entering new AgentExecutor chain...
你好，Bob！很高兴认识你。有什么我可以帮助你的吗？

> Finished chain.


{'input': 'hi! 我叫bob',
 'chat_history': [],
 'output': '你好，Bob！很高兴认识你。有什么我可以帮助你的吗？'}

In [39]:
#手动构造记忆数据
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! 我叫bob"),
            AIMessage(content="你好，Bob！很高兴认识你。有什么我可以帮助你的吗？"),
        ],
        "input": "我叫什么名字?",
    }
)



> Entering new AgentExecutor chain...
你的名字是Bob。

> Finished chain.


{'chat_history': [HumanMessage(content='hi! 我叫bob'),
  AIMessage(content='你好，Bob！很高兴认识你。有什么我可以帮助你的吗？')],
 'input': '我叫什么名字?',
 'output': '你的名字是Bob。'}

In [40]:
#使用RunnableWithMessageHistory自动构造记忆数据
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 历史记录会存储在内存中
message_history = ChatMessageHistory()

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [41]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "hi! 我是tomie"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)



> Entering new AgentExecutor chain...
你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？

> Finished chain.


{'input': 'hi! 我是tomie',
 'chat_history': [],
 'output': '你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？'}

In [42]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "我叫什么名字?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)



> Entering new AgentExecutor chain...
你的名字是Tomie。

> Finished chain.


{'input': '我叫什么名字?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？')],
 'output': '你的名字是Tomie。'}

In [43]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "LangSmith如何使用?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': '如何使用LangSmith'}`


LangSmith User Guide | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî 

{'input': 'LangSmith如何使用?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？'),
  HumanMessage(content='我叫什么名字?'),
  AIMessage(content='你的名字是Tomie。')],
 'output': 'LangSmith是一个用于LLM应用开发、监控和测试的平台。以下是一些主要的使用方式：\n\n1. **原型设计**：LangSmith支持快速实验，包括提示、模型类型、检索策略和其他参数。这对于理解模型的性能并调试其失败的地方非常重要。\n\n2. **调试**：在开发新的LLM应用时，我们建议默认启用LangSmith跟踪。\n\n3. **评估**：LangSmith允许开发者创建数据集，这些数据集是输入和参考输出的集合，可以用来对LLM应用进行测试。\n\n4. **比较视图**：当你在原型设计不同版本的应用并进行更改时，比较视图可以帮助你看到是否对初始测试用例产生了回归。\n\n5. **游乐场**：LangSmith提供了一个游乐场环境，用于快速迭代和实验。\n\n6. **生产监控和自动化**：LangSmith提供了清晰的可见性和每个LLM序列步骤的调试信息，使得识别和找出问题的根源变得更加容易。\n\n以上就是LangSmith的一些主要使用方式，希望对你有所帮助。'}

In [44]:
#调用方式
agent_with_chat_history.invoke(
    {"input": "截止目前我们都聊了什么?"},
    #注意此处session_id没有用到，因为我们没用类似redis的存储,只是用了一个变量
    config={"configurable": {"session_id": "foo_001"}},
)



> Entering new AgentExecutor chain...
我们的对话如下：

1. 你向我介绍了你的名字，你叫Tomie。
2. 我确认了你的名字是Tomie。
3. 你询问了如何使用LangSmith。
4. 我向你解释了LangSmith的主要使用方式，包括原型设计、调试、评估、比较视图、游乐场以及生产监控和自动化。

> Finished chain.


{'input': '截止目前我们都聊了什么?',
 'chat_history': [HumanMessage(content='hi! 我是tomie'),
  AIMessage(content='你好，Tomie！很高兴见到你。有什么我可以帮助你的吗？'),
  HumanMessage(content='我叫什么名字?'),
  AIMessage(content='你的名字是Tomie。'),
  HumanMessage(content='LangSmith如何使用?'),
  AIMessage(content='LangSmith是一个用于LLM应用开发、监控和测试的平台。以下是一些主要的使用方式：\n\n1. **原型设计**：LangSmith支持快速实验，包括提示、模型类型、检索策略和其他参数。这对于理解模型的性能并调试其失败的地方非常重要。\n\n2. **调试**：在开发新的LLM应用时，我们建议默认启用LangSmith跟踪。\n\n3. **评估**：LangSmith允许开发者创建数据集，这些数据集是输入和参考输出的集合，可以用来对LLM应用进行测试。\n\n4. **比较视图**：当你在原型设计不同版本的应用并进行更改时，比较视图可以帮助你看到是否对初始测试用例产生了回归。\n\n5. **游乐场**：LangSmith提供了一个游乐场环境，用于快速迭代和实验。\n\n6. **生产监控和自动化**：LangSmith提供了清晰的可见性和每个LLM序列步骤的调试信息，使得识别和找出问题的根源变得更加容易。\n\n以上就是LangSmith的一些主要使用方式，希望对你有所帮助。')],
 'output': '我们的对话如下：\n\n1. 你向我介绍了你的名字，你叫Tomie。\n2. 我确认了你的名字是Tomie。\n3. 你询问了如何使用LangSmith。\n4. 我向你解释了LangSmith的主要使用方式，包括原型设计、调试、评估、比较视图、游乐场以及生产监控和自动化。'}

# agentType更新版

![agtype.png](agtype.png)